# Parking Sign Recognition


### Steps

1. pictures of every possible sign [doesn't need words]
2. need to find python library to do the following a) Recognize the sign object b) match the sign with our database of base signs c) read the text

2a & b) opencv python (cv2)
2c) Pytesseract-ocr


# Library imports

In [1]:
# image detection
import cv2 #opencv is cv2
import numpy as np
import imutils

# reading text
import pytesseract
import PIL.Image


# Reading and Displaying Images

In [5]:
# Youtube tut: https://www.youtube.com/watch?v=T-0lZWYWE9Y&list=PLzMcBGfZo4-lUA8uGjeXhBUUzPYc6vZRn&index=7

# Loading Images
# imread has a second paramter of -1, 0 or 1
# -1: loads a colored image, neglecting transparency (default)
# 0: loads image in grayscale
# 1: includes image as it is, including transparency

imgt1 = cv2.imread('Parking Signs/Base Signs/no standing.jpg', 0) #imgt1 for image template1
img1 = cv2.imread('Parking Signs/Signs/cams house.jpg', 0) #img1, this is the image we will run template again.
img1 = cv2.resize(img1, (800,800)) # This image was way too big initially

# These next 3 lines displays the image
cv2.imshow('template1',imgt1) # you need the next two lines to run this
cv2.waitKey(0) # Waits a number of seconds until we press a key to move onto the next line, 0 indicates to wait infinitely
cv2.destroyAllWindows() # closes the image
cv2.imshow('image1',img1)
cv2.waitKey(0)
cv2.destroyAllWindows() 


# images are understood by python as numpy arrays representing 3 values per pixel [blue, green, red] where blue green red represent a number from 0-255
# the columns represent the width while the rows represent the height

# Template Matching Algorithm

In [13]:
# To recognize a sign, we can use corner detection (shi-thomasi Detector & goodFeaturesToTrack()). This may be problematic by picking up other corners in an image
# however this does allow you to specify a few paramaters to help improve accuracy. 
# Paramaters are: (input image, # of best corners to choose, confidence level, min euclidean distance between corners)

# Template matching is likely the algorithm to use. A potential issue is that the size of the template image needs to be close enough to the size in the actual image.
# In our case we are going to try and match pictures in Base Signs to picutres in Signs folder.
# This algorithm requires grayscale. Note that since we are using grayscale now instead of a 3 dimensional array [height, width, channel], its just [height, width]
# 6 different methods to template match. It is recommended to try each method and figure out which gets the best results.
# The methods are TM_CCOEFF, TM_CCOEFF_NORMED, TM_CCORR, TM_CCORR_NORMED, TM_SQDIFF, TM_SQDIFF_NORMED

methods = [cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR, cv2.TM_CCORR_NORMED, cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]

h, w = imgt1.shape

# loop through all methods and find the best one
for method in methods:
    img2 = img1.copy()
    
    result = cv2.matchTemplate(img2, imgt1, method) # this performs a convolution. In other words, it slides img2 all around imgt1 until it finds a reasonable match.
                                            # this returns a 2 dimension array telling us how confident of a match there is in each region of the image
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result) # we case about min_loc, max_loc. These are the locations with best match. Max or min is best, depending on the method.
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        location = min_loc
    else:
        location = max_loc

    bottom_right = (location[0] + w, location[1] + h)
    cv2.rectangle(img2, location, bottom_right, 255, 5) # We draw a rectangle at the location, 255 for black, line thickness 5
    cv2.imshow('Match', img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows() # Seems like method 1 is the only one that works for this example. Size of both images is very important. Need to figure out best sizing.


    # Ideally, once we draw a rectangle around the image, we crop it and feed it into pytesseract to read the text

# Canny Method of Edge Detection

In [14]:


#import numpy as np
#import cv2
#import imutils

image = cv2.imread('Parking Signs/Signs/contradictory_parkingsigns.jpeg')

def find_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(image=gray, threshold1=150, threshold2=175)
    cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    

    cv2.imshow('edged',edged) # you need the next two lines to run this
    cv2.waitKey(0) # Waits a number of seconds until we press a key to move onto the next line, 0 indicates to wait infinitely
    cv2.destroyAllWindows() # closes the image


find_edges(image)





# Attempting to Read signs using Pytesseract

In [16]:
# Read in some base signs
imgt1 = cv2.imread('Parking Signs/Base Signs/no standing.jpg', 0)
imgt2 = cv2.imread('Parking Signs/Base Signs/can park here.jpg', 0)
imgt3 = cv2.imread('Parking Signs/Base Signs/no parking anytime.png', 0)
imgt4 = cv2.imread('Parking Signs/Base Signs/no parking by time2.png', 0)

# pytesseract has a engine mode and page segmentation mode to choose for specific situations for more accuracy

# Page segmentation modes:
# 0. Orientation and script detection (OSD) only.
# 1. Automatic page segmentation with OSD.
# 2. Automatic page segmentation, but no OSD, or OCR. (not implemented)
# 3. Fully automatic page segmentation, but no OSD. (Default)
# 4. Assume a single column of text of variable sizes.
# 5. Assume a single uniform block of vertically aligned text.
# 6. Assume a single uniform block of text.
# 7. Treat the image as a single text line.
# 8. Treat the image as a single word.
# 9. Treat the image as a single word in a circle.
# 10. Treat the image as a single character.
# 11. Sparse text. Find as much text as possible in no particular order.
# 12. Sparse text with OSD.
# 13. Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.

# OCR Engine modes:
# 0. Legacy engine only.
# 1. Neural nets LSTM engine only.
# 2. Legacy + LSTM engines.
# 3. Default, based on what is available.

# Lets try this configuration
myconfig = r"--psm 11 --oem 3"

# Here is a function where we can draw boxes around the characters recognized by tesseract
def box_char(img):
    boxes = pytesseract.image_to_boxes(img)
    h, w, z = img.shape
    for box in boxes.splitlines():
        box = box.split(" ")
        img = cv2.rectangle(img, int(box[1]), h - int(box[2]), int(box[3]), h - int(box[4]), (0,255,0), 2)
    cv2.WaitKey(0)
    cv2.destroyAllWindows()


print("image1")
text = pytesseract.image_to_string(imgt1, config = myconfig)
print(text)
cv2.imshow('template1',imgt1)
cv2.waitKey(0)
cv2.destroyAllWindows()
box_char(imgt1)


print("image2")
text2 = pytesseract.image_to_string(imgt2, config = myconfig)
print(text2)
cv2.imshow('template1',imgt2)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("image3")
text3 = pytesseract.image_to_string(imgt3, config = myconfig)
print(text3)
cv2.imshow('template1',imgt3)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("image4")
text4 = pytesseract.image_to_string(imgt4, config = myconfig)
print(text4)
cv2.imshow('template1',imgt4)
cv2.waitKey(0)
cv2.destroyAllWindows()


image1
S

M- PM

MON-FRI

